## **Librerías**

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_columns = False

In [4]:
import matplotlib.pyplot as plt

In [5]:
from sklearn.datasets import load_iris

from sklearn.cluster import KMeans as SK_KMeans

In [6]:
from K_Means_Scratch import KMeans_Scratch

## **Data**

In [7]:
data = load_iris()

In [8]:
X = data.data

## **Parametros**

In [9]:
n_clusters = 3
max_iter = 300
random_state = 13

## **Creación del Modelo**

### **Calculo Inicial de los Centroides**

In [10]:
def initialize_centroids(X, n_clusters, random_state):
    # Inicializamos nuestra semilla
    np.random.seed(random_state)

    # Asignamos un valor aleatorio a las observaciones de nuestro dataframe
    random_idx = np.random.permutation(X.shape[0])

    # Traemos los primeras registros dependiendo de nuestra cantidad de clusters, estos seran los centroides
    centroids = X[random_idx[:n_clusters]]

    return centroids

In [11]:
centroids = initialize_centroids(X, n_clusters, random_state)
centroids

array([[5.7, 2.8, 4.5, 1.3],
       [5.6, 2.9, 3.6, 1.3],
       [5.1, 3.7, 1.5, 0.4]])

### **Calculo de las Distancias**

In [12]:
def compute_distances(X, centroids):
    # Calculamos la distancia entre cada observación contra los centroides
    distances = np.sqrt(
        ((X - centroids[:, np.newaxis]) ** 2).sum(axis=2)
    )

    return distances

In [13]:
distances = compute_distances(X, centroids)

### **Actualización de los centroides**

In [14]:
# Asignamos el cluster en función de la distancia minima
labels = np.argmin(distances, axis=0)

In [15]:
def update_centroids(X, labels, n_clusters):
    # Actualizamos el valor de nuestros centroides con el valor promedio del cluster
    centroids = [X[labels == i].mean(axis=0) for i in range(n_clusters)]
    centroids = np.array(centroids)

    return centroids

In [16]:
update_centroids(X, labels, n_clusters)

array([[6.40116279, 2.93488372, 5.10697674, 1.7627907 ],
       [5.40714286, 2.48571429, 3.67142857, 1.14285714],
       [5.006     , 3.428     , 1.462     , 0.246     ]])

### **Suma de los errores**

In [17]:
# Calculamos el valor del error cuadratico medio
def compute_sse(X, labels, centroids):
    distances = np.sqrt(
        ((X - centroids[labels])**2).sum(axis=1)
    )

    sse = np.sum(distances**2)

    return sse

In [18]:
compute_sse(X, labels, centroids)

213.86999999999995

### **Ajuste K-Means**

In [19]:
def fit_kmeans(X, n_clusters, max_iter, random_state):
    # Punto de partida de los centroides
    centroids = initialize_centroids(X, n_clusters, random_state)

    # Para dado número de iteraciones repetimos el proceso para encontrar los centroides más óptimos 
    for i in range(max_iter):
        old_centroids = centroids

        # Calculamos la distancia entre cada observación con cada clusters
        distances = compute_distances(X, centroids)

        # Asignamos el cluster respectivo
        labels = np.argmin(distances, axis=0)

        # Actualizamos los centroides con el promedio de los clusters
        centroids = update_centroids(X, labels, n_clusters)

        # Si el valor de los centroides es igual al anterior salimos del ciclo. El modelo ha convergido 
        if np.all(centroids == old_centroids):
            break

    # Calculamos el error cuadratico medio
    inertia = compute_sse(X, labels, centroids)

    return centroids, labels, inertia 

In [20]:
centroids, labels, inertia = fit_kmeans(X, n_clusters, max_iter, random_state)

In [21]:
centroids

array([[6.85384615, 3.07692308, 5.71538462, 2.05384615],
       [5.88360656, 2.74098361, 4.38852459, 1.43442623],
       [5.006     , 3.428     , 1.462     , 0.246     ]])

In [22]:
labels

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], dtype=int64)

In [23]:
inertia

78.8556658259773

In [41]:
df_results = pd.concat([pd.DataFrame(X), pd.DataFrame(labels)], axis=1)
df_results.columns = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width', 'labels']

df_results

,sepal length (cm),sepal width (cm),petal length (cm),petal width,labels
0,5.1,3.5,1.4,0.2,2
1,4.9,3.0,1.4,0.2,2
2,4.7,3.2,1.3,0.2,2
3,4.6,3.1,1.5,0.2,2
4,5.0,3.6,1.4,0.2,2
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,0
148,6.2,3.4,5.4,2.3,0
